In [ ]:
!pip install googlemaps pandas

In [ ]:
import pandas as pd
import googlemaps
from datetime import datetime
import time
import math
import json
# ============ CONFIGURACIÓN ============

API_KEY = "API_KEY"   # <-- REEMPLAZA ESTO
CSV_PATH = "/content/raw_pixels.csv"
OUTPUT_CSV_PATH = "/content/input_pixels.xlsx"
SLEEP_BETWEEN_CALLS = 0.1  # en segundos

# ======================================

gmaps = googlemaps.Client(key=API_KEY)

In [ ]:
def get_travel_time_from_google(lat, lon, client):
    """
    Truco:
    - Se hace un pequeño desplazamiento de ~20 m hacia el norte
    - Google entrega travel_time con y sin tráfico
    - Luego se escala a la distancia interna del pixel
    """
    # ~20 metros en grados
    delta = 50 / 111320

    origin = (lat, lon)
    destination = (lat + delta, lon)

    result = client.distance_matrix(
        origins=[origin],
        destinations=[destination],
        mode="driving",
        departure_time=datetime.now(),
        traffic_model="best_guess"
    )

    element = result["rows"][0]["elements"][0]

    if element.get("status") != "OK":
        return None, None

    # tiempos en segundos
    duration = element["duration"]["value"]
    duration_traffic = element.get("duration_in_traffic", {}).get("value", duration)

    # distancia real que mide Google (en metros)
    dist_m = element["distance"]["value"]
    dist_km = dist_m / 1000.0

    return duration, duration_traffic, dist_km


def compute_internal_speed(row, client):
    lat = row["lat"]
    lon = row["lon"]
    area = row["area_surface"]

    # lado del cuadrado
    lado_km = math.sqrt(area)

    # diagonal = distancia interna máxima
    diagonal_km = lado_km * math.sqrt(2)

    duration, duration_traffic, dist_small_km = get_travel_time_from_google(lat, lon, client)

    if duration is None or duration_traffic is None or duration == 0 or duration_traffic == 0:
        return '{"large": 27.872064777327935, "small": 29.759654178674353}'

    # velocidad aproximada de Google para 20 m → km/h
    speed_no_traffic = (dist_small_km / (duration / 3600))
    speed_with_traffic = (dist_small_km / (duration_traffic / 3600))

    # Escalamos a la distancia interna del pixel
    large_speed_no_traffic = speed_no_traffic
    large_speed_with_traffic = speed_with_traffic

    result = {
        "large": large_speed_with_traffic,
        "small": large_speed_no_traffic
    }
    return json.dumps(result)


# ============================
# EJECUCIÓN
# ============================

df = pd.read_csv(CSV_PATH)

df["speed_intra_stop"] = df.apply(lambda row: compute_internal_speed(row, gmaps), axis=1)
df.head()

In [ ]:
df.to_csv(OUTPUT_CSV_PATH, index=False)
